In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [2]:
# 데이터 불러오기 
sal_23 = pd.read_csv('2023_연봉_statiz.csv')
sal_24 = pd.read_csv('2024_연봉_statiz.csv')
sal_25 = pd.read_csv('2025_연봉_statiz.csv')

rookie_24 = pd.read_csv('kbo_draft_2023_전체_숫자.csv') # 신인 선수 계약금

sal_for_23 = pd.read_csv('2023 선수단 등록 현황 자료.csv', encoding='euc-kr') # 외국인 선수
sal_for_24 = pd.read_csv('2024 선수단 등록 현황 자료.csv', encoding='euc-kr')
sal_for_25 = pd.read_csv('2025 선수단 등록 현황 자료.csv', encoding='euc-kr')

### 2024년도 연봉 데이터 합치기 

In [4]:
display(sal_24.head(5))
display(rookie_24.head(5))

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
0,류현진,250000,4.56,54775,한화 이글스,10590
1,채은성,100000,1.40,71340,한화 이글스,11215
2,페냐,65000,0.35,187034,한화 이글스,15146
3,페라자,60000,2.90,20686,한화 이글스,16121
4,최재훈,60000,1.68,35760,한화 이글스,10170


,팀,선수,계약금,포지션
0,한화 이글스,김서현,500000000,투수
1,한화 이글스,문현빈,150000000,타자
2,한화 이글스,이민준,110000000,타자
3,한화 이글스,김관우,90000000,투수
4,한화 이글스,김해찬,70000000,타자


In [5]:
# 1. 연봉 단위 통일 (만원) 
rookie_24.rename(columns={'계약금': '계약금(만원)'}, inplace=True)
rookie_24['계약금(만원)'] = pd.to_numeric(rookie_24['계약금(만원)'], errors='coerce') / 10000

In [6]:
display(sal_24.head(5))
display(rookie_24.head(5))

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
0,류현진,250000,4.56,54775,한화 이글스,10590
1,채은성,100000,1.40,71340,한화 이글스,11215
2,페냐,65000,0.35,187034,한화 이글스,15146
3,페라자,60000,2.90,20686,한화 이글스,16121
4,최재훈,60000,1.68,35760,한화 이글스,10170


,팀,선수,계약금(만원),포지션
0,한화 이글스,김서현,50000.0,투수
1,한화 이글스,문현빈,15000.0,타자
2,한화 이글스,이민준,11000.0,타자
3,한화 이글스,김관우,9000.0,투수
4,한화 이글스,김해찬,7000.0,타자


In [7]:
# 2. 선수 이름과 팀 이름 모두 일치하는 데이터가 있는지 확인
dup_list = sal_24[sal_24.duplicated(subset=['선수', '팀'], keep=False)][['선수', '팀']].drop_duplicates()
dup_list

,선수,팀
439,이승현,삼성 라이온즈
440,김태훈,삼성 라이온즈


In [8]:
sal_24[(sal_24['선수'] == '이승현') & (sal_24['팀'] == '삼성 라이온즈')]

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
439,이승현,17000,0.62,27425,삼성 라이온즈,10527
455,이승현,7000,2.22,3154,삼성 라이온즈,14798


In [9]:
sal_24[(sal_24['선수'] == '김태훈') & (sal_24['팀'] == '삼성 라이온즈')]

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
440,김태훈,17000,0.81,20971,삼성 라이온즈,10871
472,김태훈,4100,-0.02,-247030,삼성 라이온즈,11336


### 연봉 데이터 + 신인 계약금 데이터 병합하기 

In [11]:
# 1. 선수 + 팀 기준으로 병합을 위한 키 생성
sal_24['선수_팀'] = sal_24['선수'] + '_' + sal_24['팀']
rookie_24['선수_팀'] = rookie_24['선수'] + '_' + rookie_24['팀']

# 2. rookie에서 필요한 컬럼만 추출
rookie_key = rookie_24[['선수_팀', '계약금(만원)']]

# 3. sal_24에 계약금 merge
sal_24 = sal_24.merge(rookie_key, on='선수_팀', how='left')

# 4. 연봉 + 계약금 합치기 (계약금이 없는 경우 0으로 처리)
sal_24['연봉(만원)'] = pd.to_numeric(sal_24['연봉(만원)'], errors='coerce')
sal_24['계약금(만원)'] = sal_24['계약금(만원)'].fillna(0)
sal_24['연봉(만원)'] = sal_24['연봉(만원)'] + sal_24['계약금(만원)']

# 5. 중간 컬럼 정리
sal_24 = sal_24.drop(columns=['선수_팀', '계약금(만원)'])

In [12]:
sal_24

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
0,류현진,250000.0,4.56,54775,한화 이글스,10590
1,채은성,100000.0,1.40,71340,한화 이글스,11215
2,페냐,65000.0,0.35,187034,한화 이글스,15146
3,페라자,60000.0,2.90,20686,한화 이글스,16121
4,최재훈,60000.0,1.68,35760,한화 이글스,10170
...,...,...,...,...,...,...
850,박준형,3000.0,-0.25,-12108,키움 히어로즈,14211
851,박윤성,12000.0,0.04,72014,키움 히어로즈,15475
852,서유신,6000.0,-,-,키움 히어로즈,15483
853,이명기,3000.0,-,-,키움 히어로즈,14212


In [13]:
# 영어 이름 지우기 (한글로 작성) 
sal_for_23['성명'] = sal_for_23['성명'].str.replace(r'[\n\\(].*$', '', regex=True).str.strip()
sal_for_24['성명'] = sal_for_24['성명'].str.replace(r'[\n\\(].*$', '', regex=True).str.strip()
sal_for_25['성명'] = sal_for_25['성명'].str.replace(r'[\n\\(].*$', '', regex=True).str.strip()

# 성 지우기 (연봉 데이터 파일과 선수이름 맞추는 과정) 
# 이름 문자열의 마지막 단어만 추출
sal_for_23['성명'] = sal_for_23['성명'].str.split().str[-1]
sal_for_24['성명'] = sal_for_24['성명'].str.split().str[-1]
sal_for_25['성명'] = sal_for_25['성명'].str.split().str[-1]

In [14]:
# 구단 명 맞추기 
team_name_map = {
    '한화': '한화 이글스',
    '두산': '두산 베어스',
    'KIA': 'KIA 타이거즈',
    'KT': 'KT 위즈',
    'LG': 'LG 트윈스',
    '롯데': '롯데 자이언츠',
    'NC': 'NC 다이노스',
    '삼성': '삼성 라이온즈',
    'SSG': 'SSG 랜더스',
    '키움': '키움 히어로즈',
}

sal_for_23['구단'] = sal_for_23['구단'].replace(team_name_map)
sal_for_24['구단'] = sal_for_24['구단'].replace(team_name_map)
sal_for_25['구단'] = sal_for_25['구단'].replace(team_name_map)

In [15]:
# 연봉($)를 연봉(만원)으로 변환하기 
sal_for_23['연봉(만원)'] = sal_for_23['연봉($)'] * 1300 / 10000
sal_for_24['연봉(만원)'] = sal_for_24['연봉($)'] * 1300 / 10000
sal_for_25['연봉(만원)'] = sal_for_25['연봉($)'] * 1300 / 10000

# 계약금($)를 계약금(만원)으로 변환하기 
sal_for_23['계약금(만원)'] = sal_for_23['계약금($)'] * 1300 / 10000

sal_for_24['계약금($)'] = pd.to_numeric(sal_for_24['계약금($)'], errors='coerce')
sal_for_24['계약금(만원)'] = sal_for_24['계약금($)'] * 1300 / 10000

sal_for_25['계약금($)'] = pd.to_numeric(sal_for_25['계약금($)'], errors='coerce')
sal_for_25['계약금(만원)'] = sal_for_25['계약금($)'] * 1300 / 10000

In [16]:
foreign_salary_2023 = sal_for_23
foreign_salary_2024 = sal_for_24
foreign_salary_2025 = sal_for_25

# 기존 변수 삭제 
del sal_for_23, sal_for_24, sal_for_25

In [17]:
foreign_salary_2023 = foreign_salary_2023[foreign_salary_2023['입단연도'] == 2023]
display(foreign_salary_2023[['구단', '성명', '입단연도', '계약금(만원)', '연봉(만원)']].head(5))

,구단,성명,입단연도,계약금(만원),연봉(만원)
0,SSG 랜더스,맥카티,2023,0.0,78000.0
1,SSG 랜더스,로메로,2023,0.0,104000.0
2,SSG 랜더스,에레디아,2023,0.0,117000.0
4,키움 히어로즈,후라도,2023,0.0,110500.0
8,LG 트윈스,딘,2023,13000.0,52000.0


In [18]:
# 1. foreign 데이터에 최종 연봉 생성
foreign_salary_2023['최종연봉'] = (
    foreign_salary_2023['계약금(만원)'].fillna(0) +
    foreign_salary_2023['연봉(만원)'].fillna(0)
)
# foreign_salary_2023

In [19]:
# 2. 기존 컬럼 제거하고 최종연봉 → 연봉(만원) 으로 이름 변경
foreign_salary_2023.drop(columns=['계약금(만원)', '연봉(만원)'], inplace=True)
foreign_salary_2023.rename(columns={'최종연봉': '연봉(만원)'}, inplace=True)

In [20]:
# 3. 칼럼명 통일하기
foreign_salary_2023.rename(columns={ '구단': '팀', '성명': '선수'}, inplace=True)
foreign_salary_2024.rename(columns={ '구단': '팀', '성명': '선수'}, inplace=True)
foreign_salary_2025.rename(columns={ '구단': '팀', '성명': '선수'}, inplace=True)

foreign_salary_2023

,팀,선수,위치,연령(세),신장(cm),체중(kg),계약금($),연봉($),입단연도,국적,연봉(만원)
0,SSG 랜더스,맥카티,투수,29,173,83,0,600000,2023,미국,78000.0
1,SSG 랜더스,로메로,투수,33,190,105,0,800000,2023,도미니카\n공화국,104000.0
2,SSG 랜더스,에레디아,외야수,33,178,88,0,900000,2023,쿠바,117000.0
4,키움 히어로즈,후라도,투수,28,188,109,0,850000,2023,파나마,110500.0
8,LG 트윈스,딘,외야수,31,183,97,100000,400000,2023,미국,65000.0
10,KT 위즈,슐서,투수,30,188,88,60000,650000,2023,미국,92300.0
12,KIA 타이거즈,메디나,투수,28,185,84,36000,300000,2023,도미니카\n공화국,43680.0
13,KIA 타이거즈,앤더슨,투수,30,193,102,100000,600000,2023,미국,91000.0
15,NC 다이노스,페디,투수,31,193,92,200000,800000,2023,미국,130000.0
16,NC 다이노스,와이드너,투수,30,183,92,145000,598000,2023,미국,96590.0


In [21]:
display(sal_23.head(5))

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
0,채은성,180000,2.36,76221,한화 이글스,11215
1,최재훈,80000,3.90,20492,한화 이글스,10170
2,스미스,70000,0.09,756875,한화 이글스,15540
3,오그레디,70000,-0.87,-80365,한화 이글스,15541
4,이태양,66000,1.44,45702,한화 이글스,10609


In [22]:
# 1. merge: 외국인 선수의 연봉(만원)만 가져옴
sal_23_merged = sal_23.merge(
    foreign_salary_2023[['팀', '선수', '연봉(만원)']],
    on=['팀', '선수'],
    how='left',
    suffixes=('', '_new')
)

sal_23_merged

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연봉(만원)_new
0,채은성,180000,2.36,76221,한화 이글스,11215,NaN
1,최재훈,80000,3.90,20492,한화 이글스,10170,NaN
2,스미스,70000,0.09,756875,한화 이글스,15540,104000.0
3,오그레디,70000,-0.87,-80365,한화 이글스,15541,91000.0
4,이태양,66000,1.44,45702,한화 이글스,10609,NaN
...,...,...,...,...,...,...,...
857,이호열,3000,-,-,키움 히어로즈,15480,NaN
858,변헌성,3000,-,-,키움 히어로즈,15481,NaN
859,안겸,3000,-,-,키움 히어로즈,15482,NaN
860,서유신,3000,-,-,키움 히어로즈,15483,NaN


In [23]:
# 2. 외국인 연봉이 있으면 덮어쓰기, 없으면 기존 연봉 유지
sal_23_merged['연봉(만원)'] = sal_23_merged['연봉(만원)_new'].combine_first(sal_23_merged['연봉(만원)'])
sal_23_merged

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연봉(만원)_new
0,채은성,180000,2.36,76221,한화 이글스,11215,NaN
1,최재훈,80000,3.90,20492,한화 이글스,10170,NaN
2,스미스,104000.0,0.09,756875,한화 이글스,15540,104000.0
3,오그레디,91000.0,-0.87,-80365,한화 이글스,15541,91000.0
4,이태양,66000,1.44,45702,한화 이글스,10609,NaN
...,...,...,...,...,...,...,...
857,이호열,3000,-,-,키움 히어로즈,15480,NaN
858,변헌성,3000,-,-,키움 히어로즈,15481,NaN
859,안겸,3000,-,-,키움 히어로즈,15482,NaN
860,서유신,3000,-,-,키움 히어로즈,15483,NaN


In [24]:
# 3. 외국인 연봉 컬럼 제거
sal_23_merged.drop(columns=['연봉(만원)_new'], inplace=True)
sal_23 = sal_23_merged.copy()
sal_23

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
0,채은성,180000,2.36,76221,한화 이글스,11215
1,최재훈,80000,3.90,20492,한화 이글스,10170
2,스미스,104000.0,0.09,756875,한화 이글스,15540
3,오그레디,91000.0,-0.87,-80365,한화 이글스,15541
4,이태양,66000,1.44,45702,한화 이글스,10609
...,...,...,...,...,...,...
857,이호열,3000,-,-,키움 히어로즈,15480
858,변헌성,3000,-,-,키움 히어로즈,15481
859,안겸,3000,-,-,키움 히어로즈,15482
860,서유신,3000,-,-,키움 히어로즈,15483


In [25]:
foreign_salary_2024 = foreign_salary_2024[foreign_salary_2024['입단연도'].astype(str).str.contains('2024')]
foreign_salary_2024[['선수', '팀', '입단연도', '계약금(만원)', '연봉(만원)']]

,선수,팀,입단연도,계약금(만원),연봉(만원)
1,엔스,LG 트윈스,2024,39000.0,78000.0
6,더거,SSG 랜더스,2024,13000.0,84500.0
9,카스타노,NC 다이노스,2024,16900.0,67600.0
10,하트,NC 다이노스,2024,26000.0,65000.0
11,데이비슨,NC 다이노스,2024,18200.0,72800.0
14,라모스,두산 베어스,2022(KT)\n2024(두산),6500.0,71500.0
15,크로우,KIA 타이거즈,2024,26000.0,78000.0
16,네일,KIA 타이거즈,2024,26000.0,45500.0
20,레이예스,롯데 자이언츠,2024,13000.0,78000.0
21,레예스,삼성 라이온즈,2024,13000.0,65000.0


In [26]:
# 1. foreign 데이터에 최종 연봉 생성
foreign_salary_2024['최종연봉'] = (
    foreign_salary_2024['계약금(만원)'].fillna(0) +
    foreign_salary_2024['연봉(만원)'].fillna(0)
)

# 2. 기존 컬럼 제거하고 최종연봉 → 연봉(만원) 으로 이름 변경
foreign_salary_2024.drop(columns=['계약금(만원)', '연봉(만원)'], inplace=True)
foreign_salary_2024.rename(columns={'최종연봉': '연봉(만원)'}, inplace=True)

foreign_salary_2024

,팀,선수,위치,연령(세),신장(cm),체중(kg),계약금($),연봉($),입단연도,국적,연봉(만원)
1,LG 트윈스,엔스,투수,33,185,95,300000.0,600000,2024,미국,117000.0
6,SSG 랜더스,더거,투수,29,183,83,100000.0,650000,2024,미국,97500.0
9,NC 다이노스,카스타노,투수,30,190,104,130000.0,520000,2024,미국,84500.0
10,NC 다이노스,하트,투수,32,196,90,200000.0,500000,2024,미국,91000.0
11,NC 다이노스,데이비슨,내야수,31,190,104,140000.0,560000,2024,미국,91000.0
14,두산 베어스,라모스,외야수,32,183,97,50000.0,550000,2022(KT)\n2024(두산),푸에르토리코,78000.0
15,KIA 타이거즈,크로우,투수,30,185,108,200000.0,600000,2024,미국,104000.0
16,KIA 타이거즈,네일,투수,31,193,83,200000.0,350000,2024,미국,71500.0
20,롯데 자이언츠,레이예스,외야수,30,196,87,100000.0,600000,2024,베네수엘라,91000.0
21,삼성 라이온즈,레예스,투수,28,198,113,100000.0,500000,2024,도미니카,78000.0


In [27]:
# 1. merge: 외국인 선수의 연봉(만원)만 가져옴
sal_24_merged = sal_24.merge(
    foreign_salary_2024[['팀', '선수', '연봉(만원)']],
    on=['팀', '선수'],
    how='left',
    suffixes=('', '_new')
)

# 2. 외국인 연봉이 있으면 덮어쓰기, 없으면 기존 연봉 유지
sal_24_merged['연봉(만원)'] = sal_24_merged['연봉(만원)_new'].combine_first(sal_24_merged['연봉(만원)'])

# 3. 외국인 연봉 컬럼 제거
sal_24_merged.drop(columns=['연봉(만원)_new'], inplace=True)
sal_24 = sal_24_merged.copy()
sal_24

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
0,류현진,250000.0,4.56,54775,한화 이글스,10590
1,채은성,100000.0,1.40,71340,한화 이글스,11215
2,페냐,65000.0,0.35,187034,한화 이글스,15146
3,페라자,104000.0,2.90,20686,한화 이글스,16121
4,최재훈,60000.0,1.68,35760,한화 이글스,10170
...,...,...,...,...,...,...
850,박준형,3000.0,-0.25,-12108,키움 히어로즈,14211
851,박윤성,12000.0,0.04,72014,키움 히어로즈,15475
852,서유신,6000.0,-,-,키움 히어로즈,15483
853,이명기,3000.0,-,-,키움 히어로즈,14212


In [28]:
foreign_salary_2025 = foreign_salary_2025[foreign_salary_2025['입단연도'] == 2025]
foreign_salary_2025[['선수', '팀', '입단연도', '계약금(만원)', '연봉(만원)']]

,선수,팀,입단연도,계약금(만원),연봉(만원)
1,올러,KIA 타이거즈,2025,26000.0,78000.0
2,위즈덤,KIA 타이거즈,2025,26000.0,104000.0
4,후라도,삼성 라이온즈,2025,39000.0,91000.0
7,치리노스,LG 트윈스,2025,26000.0,104000.0
9,어빈,두산 베어스,2025,26000.0,104000.0
10,로그,두산 베어스,2025,13000.0,91000.0
11,케이브,두산 베어스,2025,26000.0,104000.0
13,헤이수스,KT 위즈,2025,26000.0,104000.0
15,화이트,SSG 랜더스,2025,NaN,130000.0
18,데이비슨,롯데 자이언츠,2025,45500.0,65000.0


In [29]:
# 1. foreign 데이터에 최종 연봉 생성
foreign_salary_2025['최종연봉'] = (
    foreign_salary_2025['계약금(만원)'].fillna(0) +
    foreign_salary_2025['연봉(만원)'].fillna(0)
)

# 2. 기존 컬럼 제거하고 최종연봉 → 연봉(만원) 으로 이름 변경
foreign_salary_2025.drop(columns=['계약금(만원)', '연봉(만원)'], inplace=True)
foreign_salary_2025.rename(columns={'최종연봉': '연봉(만원)'}, inplace=True)

foreign_salary_2025

,팀,선수,위치,연령(세),신장(cm),체중(kg),계약금($),연봉($),입단연도,국적,연봉(만원)
1,KIA 타이거즈,올러,투수,32,193,102,200000.0,600000,2025,미국,104000.0
2,KIA 타이거즈,위즈덤,내야수,35,188,99,200000.0,800000,2025,미국,130000.0
4,삼성 라이온즈,후라도,투수,30,188,109,300000.0,700000,2025,파나마,130000.0
7,LG 트윈스,치리노스,투수,33,188,102,200000.0,800000,2025,베네수엘라,130000.0
9,두산 베어스,어빈,투수,32,193,102,200000.0,800000,2025,미국,130000.0
10,두산 베어스,로그,투수,30,183,84,100000.0,700000,2025,미국,104000.0
11,두산 베어스,케이브,외야수,34,183,90,200000.0,800000,2025,미국,130000.0
13,KT 위즈,헤이수스,투수,30,192,104,200000.0,800000,2025,베네수엘라,130000.0
15,SSG 랜더스,화이트,투수,32,190,95,NaN,1000000,2025,미국,130000.0
18,롯데 자이언츠,데이비슨,투수,30,188,97,350000.0,500000,2025,미국,110500.0


In [30]:
# 1. merge: 외국인 선수의 연봉(만원)만 가져옴
sal_25_merged = sal_25.merge(
    foreign_salary_2025[['팀', '선수', '연봉(만원)']],
    on=['팀', '선수'],
    how='left',
    suffixes=('', '_new')
)

# 2. 외국인 연봉이 있으면 덮어쓰기, 없으면 기존 연봉 유지
sal_25_merged['연봉(만원)'] = sal_25_merged['연봉(만원)_new'].combine_first(sal_25_merged['연봉(만원)'])

# 3. 외국인 연봉 컬럼 제거
sal_25_merged.drop(columns=['연봉(만원)_new'], inplace=True)
sal_25 = sal_25_merged.copy()
sal_25

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid
0,류현진,200000,2.77,72322,한화 이글스,10590
1,폰세,130000.0,5.86,19116,한화 이글스,16313
2,플로리얼,97500.0,1.96,50070,한화 이글스,16312
3,엄상백,90000,-0.11,-783140,한화 이글스,11318
4,와이스,84000,3.42,24584,한화 이글스,16153
...,...,...,...,...,...,...
841,김주훈,3000,-,-,키움 히어로즈,16129
842,서유신,3000,-0.24,-12551,키움 히어로즈,15483
843,박성빈,3000,-,-,키움 히어로즈,15479
844,이우석,1500,-,-,키움 히어로즈,11360


In [31]:
sal_23['연도'] = 2023
sal_24['연도'] = 2024
sal_25['연도'] = 2025

In [32]:
display(sal_23.head(6))
display(sal_24.head(6))
display(sal_25.head(6))

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연도
0,채은성,180000,2.36,76221,한화 이글스,11215,2023
1,최재훈,80000,3.90,20492,한화 이글스,10170,2023
2,스미스,104000.0,0.09,756875,한화 이글스,15540,2023
3,오그레디,91000.0,-0.87,-80365,한화 이글스,15541,2023
4,이태양,66000,1.44,45702,한화 이글스,10609,2023
5,페냐,55000,4.20,13083,한화 이글스,15146,2023


,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연도
0,류현진,250000.0,4.56,54775,한화 이글스,10590,2024
1,채은성,100000.0,1.40,71340,한화 이글스,11215,2024
2,페냐,65000.0,0.35,187034,한화 이글스,15146,2024
3,페라자,104000.0,2.90,20686,한화 이글스,16121,2024
4,최재훈,60000.0,1.68,35760,한화 이글스,10170,2024
5,이태양,50000.0,-0.29,-170710,한화 이글스,10609,2024


,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연도
0,류현진,200000,2.77,72322,한화 이글스,10590,2025
1,폰세,130000.0,5.86,19116,한화 이글스,16313,2025
2,플로리얼,97500.0,1.96,50070,한화 이글스,16312,2025
3,엄상백,90000,-0.11,-783140,한화 이글스,11318,2025
4,와이스,84000,3.42,24584,한화 이글스,16153,2025
5,채은성,60000,1.73,34667,한화 이글스,11215,2025


In [33]:
player_sal_all = pd.concat([sal_23, sal_24, sal_25], ignore_index=True)
player_sal_all

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연도
0,채은성,180000,2.36,76221,한화 이글스,11215,2023
1,최재훈,80000,3.90,20492,한화 이글스,10170,2023
2,스미스,104000.0,0.09,756875,한화 이글스,15540,2023
3,오그레디,91000.0,-0.87,-80365,한화 이글스,15541,2023
4,이태양,66000,1.44,45702,한화 이글스,10609,2023
...,...,...,...,...,...,...,...
2558,김주훈,3000,-,-,키움 히어로즈,16129,2025
2559,서유신,3000,-0.24,-12551,키움 히어로즈,15483,2025
2560,박성빈,3000,-,-,키움 히어로즈,15479,2025
2561,이우석,1500,-,-,키움 히어로즈,11360,2025


In [34]:
# 결측치 제거 (5개) 
player_sal_all = player_sal_all.dropna()

In [35]:
# 1. 문자열 전처리 + 숫자형 변환
player_sal_all['연봉(만원)'] = (
    player_sal_all['연봉(만원)']
    .astype(str)                      # object → str
    .str.replace(',', '', regex=False)  # 쉼표 제거
    .str.strip()                     # 공백 제거
    .replace(['', '-', 'nan'], '0')  # 이상값 → '0'
    .astype(float)                   # 숫자형으로 변환
)

In [38]:
player_sal_all

,선수,연봉(만원),WAR,WAR당 연봉,팀,pid,연도
0,채은성,180000.0,2.36,76221,한화 이글스,11215,2023
1,최재훈,80000.0,3.90,20492,한화 이글스,10170,2023
2,스미스,104000.0,0.09,756875,한화 이글스,15540,2023
3,오그레디,91000.0,-0.87,-80365,한화 이글스,15541,2023
4,이태양,66000.0,1.44,45702,한화 이글스,10609,2023
...,...,...,...,...,...,...,...
2558,김주훈,3000.0,-,-,키움 히어로즈,16129,2025
2559,서유신,3000.0,-0.24,-12551,키움 히어로즈,15483,2025
2560,박성빈,3000.0,-,-,키움 히어로즈,15479,2025
2561,이우석,1500.0,-,-,키움 히어로즈,11360,2025


In [40]:
# 저장 
player_sal_all.to_csv('선수 연봉 데이터(연봉).csv', index=False, encoding='utf-8-sig')

In [37]:
10/0

ZeroDivisionError: division by zero

In [ ]:
# 2. 등급 구간 정의
bins = [0, 5000, 10000, 30000, 100000, float('inf')]
labels = ['E', 'D', 'C', 'B', 'A']

# 3. 연봉등급 컬럼 생성
player_sal_all['연봉등급'] = pd.cut(player_sal_all['연봉(만원)'], bins=bins, labels=labels, right=False)

In [ ]:
# 4. 확인
player_sal_all= player_sal_all[['팀', '선수', 'WAR', '연봉등급', 'pid', '연도']]
player_sal_all

In [ ]:
# 저장 
player_sal_all.to_csv('선수 연봉 데이터(등급).csv', index=False, encoding='utf-8-sig')